In this notebook, we'll work through an example of portfolio optimization. The exercises we do are related to the work by Harry Markowitz, a UChicago grad and winner of the Nobel prize in economics in 1990.

In [ ]:
# always import all necessary libraries at the top of the file
import numpy as np
import scipy.optimize
from matplotlib import pyplot as plt
import pandas as pd

# Data

In this folder, there should be a file called `dataAssets.csv`. Make sure that you have this file. Run the code below to load the data into numpy arrays.

In the matrix of prices, each row corresponds to a day. The columns correspond, (in order,) to the S&P 500 index, USD index, crude oil index, HYG index, and U.S. 10-yr Treasury index.

In [ ]:
data_assets = pd.read_csv('./dataAssets.csv', parse_dates=['date'])

In [ ]:
data_assets.head()

In [ ]:
# Don't worry about the Pandas details here
prices = data_assets.iloc[:,1:].values
dates = data_assets.iloc[:,0].values

# Portfolio

 - (1) Calculate log return rates using the given price data (log of the gross returns). Multiply by 100 so that the returns are in percetages.

In [ ]:
rets = np.log(prices[1:,:]/prices[:-1,:]) * 100

- (2) Form a portfolio of data, equally split between the S&P 500 and the Oil Index.

In [ ]:
portfolio = 0.5 * rets[:,0] + 0.5 * rets[:,2]

- (3) Calculate the 20th percentile of the return history on this portfolio. (Use `numpy.percentile`. Be sure to specify in terms of percetages---between 0 and 100.)

In [ ]:
percent = 30
np.percentile(portfolio, percent)

 - (4) Use `matplotlib.pyplot.hist` to create a histogram of the returns on this portfolio. Also plot a separate figure with a histogram of the returns for the S&P 500 and for Oil on the same plot. Use labels to distinguish them. You should probably plot Oil first, or at least set the alpha (transparancy) on each plot to something like 0.5. Be sure to include a legend.

In [ ]:
plt.hist(portfolio);

In [ ]:
plt.hist(rets[:,2], alpha=.5, label='Oil');
plt.hist(rets[:,0], alpha=.5, label='S&P 500');
plt.legend()

# Mean-Variance Efficient Portfolios


In this application, we replicate the Markowitz model of portfolio optimization. This model is a model of investor behavior. It models an investor you prefers a portfolio that has higher expected returns and lower variance, *ceteris paribus*. The result of this assumption is that investors (1) choose diversified portfolios and (2) that there exist on a "mean-variance efficient frontier" along which investors choose their portfolios. That is, there are some portfolios that consist of some combinations of assets that are strictly dominated by other combinations of assets. This model  formalizes the notion that an investor should not "put all of his/her eggs in one basket" as well as  the tradeoff between risk and return.

(Note: There are other important lessons that we can learn from this model, but they are beyond the scope of this class. For one, you may search Google for the concept of the "two-portfolio separation theorem." Also note that we are excluding any "risk-free" short-term assets from our analysis here. Also note that Markowitz won the Nobel prize in economics in 1990 for this analysis.)

Suppose that $\mu$ is a column vector of the means of each asset and suppose that $\Sigma$ is the associated variance-covariance matrix. Now, suppose that we create a column vector of weights $w$ that tells us the fraction of our wealth that we invest in each asset. Then, the formulas in matrix notation for the mean and variance of the resulting portfolio are
$$
\mu_p = \mu' w
$$
and
$$
\sigma_p^2 = w' \Sigma w
$$
respectively.


### Global Minimum Variance Portfolio

We'll first start by finding a particular portfolio called the "global minimum variance" portfolio (assuming the absence of a risk-free asset--why?). 

**Discussion** 

 * What is a risk-free asset? Why am I assuming that one doesn't exist?


Consider a portfolio allocation vector, `w`, which sums to 1 and allocates a fraction of the portfolio to each of the 5 securities. For example

    w = np.array([.2, .2, .2, .2, .2])
    
allocates 20% to each asset.

In [ ]:
# You may find that this code is helpful for viewing
# the arrays. Suppress will suppress scientific notation
# and precision sets the number of digits to print to
# the screen
np.set_printoptions(precision=2, suppress=True)

 - (1) First, calculate the covariance matrix of the assets. Save it to a variable called `Sigma`. Be sure to set the option of the covariance function to `rowvar=False`, since each column in our matrix is a variable, not each row. Alternatively, you could transpose the matrix.

 - (2)
Write an in-line function (a `lambda` function) that calculates the return variance for the allocation using the formula
$$
w' \Sigma w.
$$
Call the function `w`. That is, save the function to a variable called `w`.
Try out your new function using the following portfolio weights:

    w = np.array([1,0,0,0,0])
    w = np.array([.2, .2, .2, .2, .2])
    w = np.array([.4, 0, .2, .2, .2])

 - (3) Write a constraint function called `con`. This takes in the vector of portfolio weights `w` and returns the difference between 1 and the sum of the portfolio weights. The purpose is that if the weights sum to one, then the function returns a zero. The zero indicates that the contraint holds.

 - (4) Minimize this portfolio variance, subject to the constraint that portfolio weights add to 1. 
 
To do this, check out the available constrained optimizers listed in the Scipy documentation: https://docs.scipy.org/doc/scipy/reference/optimize.html

Use sequential least squares: `scipy.optimize.fmin_slsqp`. Set the desired accuracy to `acc=1e-09` and set the option `iprint=2` so that the optimizer will display information at each iteration of the algorithm. You will need to set an initial guess for the optimizer to start from. Set the initial guess to

    w0 = np.array([.2, .2, .2, .2, .2])
    
Save the final results into a new variable called `wstar`.

 - (5) Test your newly optimized function. Evaluate the covariance of the portfolio at `wstar` and 
    
    `wstar + np.array([0.1, 0, 0, 0, -0.1])`

and

    wstar - np.array([0.1, 0, 0, 0, -0.1])
    
The perturbation will ensure that the constraint still holds. Does `wstar` appear to be an minimum?

**Discussion:**

 * What do I mean when I say that the "perturbation will ensure that the constraint still holds"? 
 * Why is it important for this constraint to hold?

### Mean-Variance Efficient Portfolios

We now consider the case of constructing a portfolio that has minimum variance among all possible portfolios that have a "target" expected return. To do this, let's complete the following steps.

 - (1) Compute the means of the asset returns and the variance-covariance matrix. Save the means as the numpy array `mu`. Save the variance-covariance matrix as the numpy array `Sigma`. 

 - (2) Solve for the optimal portfolio that solves the following contrained minimization problem given a target expected portfolio return of $\mu_p = 0.01$.
 
\begin{align*}
\min_{w} \quad & w' \Sigma w \\
\text{s.t.} \quad & w' \mu = \mu_p \\
& w' \mathbb 1 = 1 
\end{align*}
where $\mathbb 1$ is a conforming vector of ones. For an initial guess for the optimizer, choose a portfolio of equal weights to each asset (a $5 \times 1$ vector of the value $0.2$).

Use `scipy.optimize.fmin_slsqp`. In order to incorporate two constraints, carefully read the documentation regarding the keyword argument `eqcons`.

Save the optimal set of portfolio weights as `wstar`. Calculate the variance of this optimal portfolio. Verify that the sum of the weights is numerically close to 1.

 - (3) Use `numpy.linspace` to create a vector of `N` values equally spaced from -0.01 to 2. Call this vector `mean_grid`. For each value in this mean grid with `N=100`, solve the above maximization problem with the value of the mean grid as the target expected portfolio return $\mu_p$. For each solution, record the value of the objective. That is, record the minimal portfolio return variance needed to acheive the desired expected portfolio return. Plot the two vectors that result on a plot with the x-axis labelled 'Portfolio Return Variance', the y-axis labelled 'Mean Portfolio Return', and the title of plot as 'Mean-Variance Efficient Frontier'.